In [247]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dropout
%matplotlib inline  
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from keras import layers, models

In [248]:
print(tf.__version__)

2.10.1


In [249]:
def prepareData():
    allData = pd.read_csv("data/songs_with_genre.csv", index_col=0, header=[0, 1, 2])
    genres = allData["genre_id"]
    genres_index = pd.read_csv("data/genres_index.csv", index_col=0)
    allData.drop(columns=["genre_id"], inplace=True)
    data_x = allData
    data_x.columns = range(data_x.shape[1])
    #normalizing
    mu = np.mean(data_x, axis=0)
    sigma = (np.std(data_x, axis=0))
    data_x = (data_x - mu) / sigma
    #Creating the right labels
    genres.columns = range(genres.shape[1])
    targets = OneHotEncoder().fit_transform(genres_index).toarray()
    genres[0] = genres[0] - 1
    data_y = [targets[i - 1] for i in genres[0]]
    genres[0] = data_y
    
    test = genres.copy()
    test.rename(columns={0: "oneHotVector"}, inplace=True)
    # test.shape
    split = pd.DataFrame(test["oneHotVector"].to_list(), columns=range(161))
    split.index = genres.index
    genres = split
    return data_x, genres

In [250]:
data_x, labels = prepareData()
x_train, x_rest, y_train, y_rest = train_test_split(data_x, labels, train_size=0.8)
x_test, x_valid, y_test, y_valid = train_test_split(x_rest, y_rest, train_size=0.5)
print(x_train.shape), print(len(y_train))
print(x_valid.shape), print(len(y_valid))
print(x_test.shape), print(len(y_test))

(39678, 518)
39678
(4960, 518)
4960
(4960, 518)
4960


(None, None)

In [251]:
# x_train = x_train.reshape((-1, 518, 128, 1))
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=161)


In [252]:
# Parameters

# Layers
kernel_size = 3
pool_size = 2
starting_filters = 32
activation_func_layer = 'relu'
activation_func_end = 'softmax'
padding_method = 'same'
optimizer_func = 'adam'
dropout = 0.5

# Fit
loss_function = 'categorical_crossentropy'
epochs = 10
batch_size = 102

# Halt
halt_monitor = 'val_accuracy'
halt_mod = 'max'
patience = 10

In [253]:
n_features, n_outputs = x_train.shape[1], y_train.shape[1]
CNN = models.Sequential()
CNN.add(layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(n_features, 128, 1), padding='same'))
CNN.add(layers.MaxPooling2D(pool_size=2))
CNN.add(layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
CNN.add(layers.MaxPooling2D(pool_size=2))
# CNN.add(layers.Conv1D(filters=128, kernel_size=4, activation='relu'))
# CNN.add(layers.MaxPooling1D(pool_size=2))
# CNN.add(layers.Conv1D(filters=256, kernel_size=4, activation='relu'))
# CNN.add(layers.MaxPooling1D(pool_size=3))
CNN.add(layers.Flatten())
# CNN.add(layers.Dropout(0.4))
CNN.add(layers.Dense(64, activation='relu'))
# CNN.add(layers.BatchNormalization())
CNN.add(layers.Dense(n_outputs, activation='softmax'))

In [254]:
CNN.summary()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_43 (Conv2D)          (None, 518, 128, 32)      320       
                                                                 
 max_pooling2d_43 (MaxPoolin  (None, 259, 64, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_44 (Conv2D)          (None, 257, 62, 64)       18496     
                                                                 
 max_pooling2d_44 (MaxPoolin  (None, 128, 31, 64)      0         
 g2D)                                                            
                                                                 
 flatten_23 (Flatten)        (None, 253952)            0         
                                                                 
 dense_46 (Dense)            (None, 64)              

In [255]:
# # Reshape the input data
# x_train = x_train.values.reshape((x_train.shape[0], x_train.shape[1], 1, 1))
# x_valid = x_valid.values.reshape((x_valid.shape[0], x_valid.shape[1], 1, 1))
# x_test = x_test.values.reshape((x_test.shape[0], x_test.shape[1], 1, 1))

In [256]:
# # Reshape the input data to have shape (num_samples, 518, 128, 1)
# num_samples = x_train.shape[0]
# x_train = x_train.values.reshape((num_samples, 518, 128, 1))
#
# # Do the same for the validation and test sets
# num_samples = x_valid.shape[0]
# x_valid = x_valid.values.reshape((num_samples, 518, 128, 1))
#
# num_samples = x_test.shape[0]
# x_test = x_test.values.reshape((num_samples, 518, 128, 1))

In [257]:
# Calculate the number of columns needed to reshape the data to have the desired shape
num_columns = x_train.shape[1] // (518 * 128)

# Select the first num_columns columns of the input data
x_train = x_train.values[:, :num_columns]

# Reshape the input data to have shape (num_samples, 518, 128, 1)
num_samples = x_train.shape[0]
x_train = x_train.reshape((num_samples, 518, 128, 1))

# Do the same for the validation and test sets
num_samples = x_valid.shape[0]
x_valid = x_valid.values[:, :num_columns].reshape((num_samples, 518, 128, 1))

ValueError: cannot reshape array of size 0 into shape (39678,518,128,1)

In [ ]:
CNN.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# # Calculate the number of samples in the input data
# num_samples = x_train.shape[0]
#
# # Calculate the number of columns needed to reshape the data to have the desired shape
# num_columns = 87328 // num_samples
# x_train = x_train.values.reshape((num_samples, 518, num_columns, 1))
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=161)

In [ ]:

callback = [tf.keras.callbacks.EarlyStopping(monitor='accuracy', mode="max", patience=10)]
training = CNN.fit(x_train, y_train, epochs=1000, validation_data=(x_valid, y_valid), batch_size=64, callbacks=callback)

In [ ]:
metrics_df = pd.DataFrame(training.history)
metrics_df[["loss","val_loss"]].plot();
metrics_df[["accuracy","val_accuracy"]].plot();  # The semicolon prevents certain matplotlib items from being printed.

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
y_train

In [ ]:

test_loss, test_accuracy = CNN.evaluate(x_train, y_train, verbose=2)

In [ ]:
test_accuracy